In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [53]:
df = pd.read_csv('data_agg/final_aggregated_output_no_outlier.csv', index_col=0)
df

,Time (s),Linear Acceleration x (m/s^2),Linear Acceleration y (m/s^2),Linear Acceleration z (m/s^2),Height (m),Velocity (m/s),Displacement (m),Distance From Last (m),Acceleration x (m/s^2),Acceleration y (m/s^2),Acceleration z (m/s^2),Gyroscope x (rad/s),Gyroscope y (rad/s),Gyroscope z (rad/s),Activity
Datetime_linacc,,,,,,,,,,,,,,,
2024-06-04 14:06:28.024712,5.024712,1.475718,-1.026762,-1.760057,-4.745554,0.98,0.105261,0.000000,0.147458,3.445964,9.239586,-0.027829,-0.391340,0.373098,Walking
2024-06-04 14:06:28.065210,5.065210,-0.108604,-0.699649,-3.437370,-4.745554,0.98,0.105261,0.000000,0.147458,3.445964,9.239586,-0.027829,-0.391340,0.373098,Walking
2024-06-04 14:06:28.105655,5.105655,-1.297407,-0.599205,-2.629323,-4.745554,0.98,0.105261,0.000000,0.147458,3.445964,9.239586,-0.027829,-0.391340,0.373098,Walking
2024-06-04 14:06:28.146197,5.146197,1.051061,-0.073347,-0.506624,-4.745554,0.98,0.105261,0.000000,0.147458,3.445964,9.239586,-0.027829,-0.391340,0.373098,Walking
2024-06-04 14:06:28.186665,5.186665,0.602122,-0.430895,-0.995502,-4.745554,0.98,0.105261,0.000000,0.147458,3.445964,9.239586,-0.027829,-0.391340,0.373098,Walking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-11 14:47:20.032852,296.032852,5.564073,-3.819289,3.387449,5.859561,0.00,0.004123,0.000246,13.409976,-8.886046,-0.433699,-0.054194,0.124233,0.115445,Sport
2024-06-11 14:47:20.073338,296.073338,5.564073,-3.819289,3.387449,5.859561,0.00,0.004123,0.000246,13.409976,-8.886046,-0.433699,-0.054194,0.124233,0.115445,Sport
2024-06-11 14:47:20.113847,296.113847,2.491645,2.183503,2.287658,5.859561,0.00,0.004123,0.000246,13.409976,-8.886046,-0.433699,-0.054194,0.124233,0.115445,Sport


In [61]:
def calc_means(df, cols, activity):
    means = {}
    for c in cols:
        means[c] = df[df["Activity"] == activity][c].mean()
    return means

In [64]:
calc_means(df, ["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"], "Sport")

{'Acceleration x (m/s^2)': -0.016452897533598167,
 'Acceleration y (m/s^2)': -8.498725284396185,
 'Acceleration z (m/s^2)': 1.144506047408796}

In [50]:
[col for col in df.columns if "mean" in col]

['x_mean', 'y_mean', 'z_mean']

In [62]:
from ML4QS.Python3Code.Chapter4.TemporalAbstraction import NumericalAbstraction

num_abs = NumericalAbstraction()

def calc_means(df, cols, method="slope"):
    means = {}
    for c in cols:
        means[c] = num_abs.aggregate_value(df[c].abs(), 10, method)
    return means

calc_means(df, ["Linear Acceleration x (m/s^2)", "Linear Acceleration y (m/s^2)", "Linear Acceleration z (m/s^2)"], "slope")

{'Linear Acceleration x (m/s^2)': Datetime_linacc
 2024-06-04 14:06:28.024712         NaN
 2024-06-04 14:06:28.065210         NaN
 2024-06-04 14:06:28.105655         NaN
 2024-06-04 14:06:28.146197         NaN
 2024-06-04 14:06:28.186665         NaN
                                 ...   
 2024-06-11 14:47:20.032852    0.285388
 2024-06-11 14:47:20.073338    0.279502
 2024-06-11 14:47:20.113847    0.028728
 2024-06-11 14:47:20.154319    0.081158
 2024-06-11 14:47:20.194784    0.082400
 Name: Linear Acceleration x (m/s^2), Length: 299602, dtype: float64,
 'Linear Acceleration y (m/s^2)': Datetime_linacc
 2024-06-04 14:06:28.024712         NaN
 2024-06-04 14:06:28.065210         NaN
 2024-06-04 14:06:28.105655         NaN
 2024-06-04 14:06:28.146197         NaN
 2024-06-04 14:06:28.186665         NaN
                                 ...   
 2024-06-11 14:47:20.032852    0.076827
 2024-06-11 14:47:20.073338    0.033783
 2024-06-11 14:47:20.113847   -0.021010
 2024-06-11 14:47:20.154319   